In [ ]:
### Import Libraries.

import os
import scanpy as sc
import pandas as pd
import numpy as np
import harmonypy as hm
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.axes
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
### Load Data.

os.chdir("/folder/")
adata =  ad.read_h5ad("adata.h5ad")

In [ ]:
### Main_1A. DimPlot Annotation.

my_levels = ["Neurons", "Oligodendrocytes", "OPCs", "Astrocytes", "Microglia",
             "Macrophages", "Endothelial Cells", "Mural Cells", "Fibroblasts",
             "Ependymal Cells", "Immune Cells", "Schwann Cells"]

my_colors = ["#9E8FB2", "#A8C5D9", "#E1C0C8", "#7DA3A1",
             "#C2765C", "#800020", "#FF9B6A", "#FFD3B6",
             "#C1D7AE", "#6B7AA1", "#B5AFA4", "#2C3E50"]

adata.obs["Annotation"] = adata.obs["Annotation"].astype("category")
adata.obs["Annotation"] = adata.obs["Annotation"].cat.set_categories(my_levels)
adata.obs["cell_type"] = adata.obs["Annotation"]

fig = sc.pl.umap(
    adata,
    color = "Annotation",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(40, 40)
fig.patch.set_facecolor("white")

ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_1_MC_SC_Annotation_Labels.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white')

In [ ]:
### Main_1B. Barplot Proportions: Annotation_Region.

meta_data = adata.obs.copy()
meta_data = meta_data[meta_data["Enrichment"] != "Yes"]

my_levels = ["Neurons", "Oligodendrocytes", "OPCs", "Astrocytes", "Microglia", 
             "Macrophages", "Endothelial Cells", "Mural Cells", "Fibroblasts", 
             "Ependymal Cells", "Immune Cells", "Schwann Cells"]

meta_data["Annotation"] = pd.Categorical(meta_data["Annotation"], categories = my_levels, ordered = True)

proportions = (
    meta_data
    .groupby(["Region", "Annotation"])
    .size()
    .reset_index(name = "Count")
)

proportions["Proportion"] = proportions.groupby("Region")["Count"].transform(lambda x: x / x.sum())

annotation_colors = {
    "Neurons": "#9E8FB2", "Oligodendrocytes": "#A8C5D9", "OPCs": "#E1C0C8",
    "Astrocytes": "#7DA3A1", "Microglia": "#C2765C", "Macrophages": "#D8E2DC",
    "Endothelial Cells": "#FF9B6A", "Mural Cells": "#FFD3B6",
    "Fibroblasts": "#C1D7AE", "Ependymal Cells": "#6B7AA1",
    "Immune Cells": "#B5AFA4", "Schwann Cells": "#D4C0A1"
}

g = sns.catplot(
    data = proportions,
    kind = "bar",
    x = "Annotation", y = "Proportion", hue = "Annotation",
    col = "Region", col_wrap = 2,
    palette = annotation_colors,
    height = 5, aspect = 1.2
)

for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, ha = "right", fontsize = 10)
    ax.set_xlabel("Cell Type", fontsize = 12)
    ax.set_ylabel("Proportion", fontsize = 12)
    ax.set_title(ax.get_title().replace("Region = ", ""), fontsize = 14, fontweight = "bold")

plt.subplots_adjust(top = 0.85)
g.fig.suptitle("Cell Type Proportions by Region", fontsize = 16, fontweight = "bold")

fig = plt.gcf()
fig.set_size_inches(5, 6)

plt.savefig("MC_SC_Proportions_Annotation.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Suppl_1A. Barplot mean age per Dataset per Status.

metadata_df = adata.obs[["Sample_ID", "Age", "Sex", "Dataset", "Status", "Organism"]].copy()
metadata = metadata_df.drop_duplicates()
metadata["Age"] = pd.to_numeric(metadata["Age"], errors = "coerce")

metadata_summary = metadata.groupby(["Dataset", "Status"], as_index = True).agg(
    Min_Age = ("Age", "min"),
    Mean_Age = ("Age", "mean"),
    Max_Age = ("Age", "max")
).reset_index()

metadata_summary["Mean_Age_Numeric"] = pd.to_numeric(metadata_summary["Mean_Age"], errors = "coerce")

sample_counts = metadata.groupby(["Dataset", "Status"])["Sample_ID"].count().reset_index()
sample_counts.rename(columns = {"Sample_ID": "N_Samples"}, inplace = True)

plot_data = metadata_summary.merge(sample_counts, on = ["Dataset", "Status"], how = "left")

my_levels = [
    "Bakken_2021_MC_Control", "Zemke_2023_MC_Control", "Li_2023_MC_Control_ALS",
    "Pineda_2024_MC_Control_ALS", "ONeill_2024_MC_Control_ALS_FTD", "Limone_2024_MC_Control_ALS",
    "Georgopoulou_2025_MC_Control_ALS", "Prashant_2024_MC_Control", "Velmeshev_2023_MC_Control",
    "Silleti_2023_MC_Control", "Zhang_2021_SC_Control", "Yadav_2023_SC_Control",
    "Silleti_2023_SC_Control", "Gautier_2023_SC_Control", "Zelic_2025_SC_Control_ALS",
    "Georgopoulou_2025_SC_Control_ALS", "Tansley_2022_SC_Control", "Donovan_2024_SC_Control"
]

cols = [
    "#4B9E9E", "#D97D54", "#D9B44A", "#A892B4", "#8AA96B",
    "#5067C5", "#C95D63", "#FF6F61", "#6A5ACD", "#20B2AA",
    "#6B7C85", "#77DD77", "#E07B39", "#C48B9F", "#613659",
    "#3BA7CB", "#FFD700", "#708090"
]

dataset_colors = dict(zip(my_levels, cols))

plot_data["Dataset"] = pd.Categorical(plot_data["Dataset"], categories = my_levels, ordered = True)
plot_data["Status"] = pd.Categorical(
    plot_data["Status"], categories = ["Control", "C9ALS", "sALS", "fALS", "sALS_FTD"], ordered = True
)

plot_data["Bar_Label"] = plot_data.apply(
    lambda row: "Not Mentioned" if pd.isna(row["Mean_Age_Numeric"]) else f"{row['Mean_Age_Numeric']:.1f}",
    axis=1
)
plot_data["Mean_Age_Plot"] = plot_data["Mean_Age_Numeric"].fillna(0)

sns.set(style = "whitegrid")
g = sns.catplot(
    data = plot_data,
    x = "Dataset", y = "Mean_Age_Plot",
    hue = "Dataset", col = "Status",
    kind = "bar", dodge = False,
    palette = dataset_colors,
    col_wrap = 5,
    height = 5, aspect = 1.5,
    legend = False
)

for ax in g.axes.flat:
    for container in ax.containers:
        labels = [plot_data.loc[
            (plot_data["Dataset"] == bar.get_x() + bar.get_width()/2) & 
            (plot_data["Status"] == ax.get_title().split(' = ')[1]), "Bar_Label"
        ].values[0] if len(plot_data.loc[
            (plot_data["Dataset"] == bar.get_x() + bar.get_width()/2) & 
            (plot_data["Status"] == ax.get_title().split(' = ')[1])
        ]) > 0 else "" for bar in container]
        ax.bar_label(container, labels = labels, label_type = "edge", fontsize = 8)

g.set_titles("{col_name}")
g.set_axis_labels("Dataset", "Mean Age")

for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, ha = "right")
    ax.set_facecolor("white")
    ax.grid(False)

plt.subplots_adjust(top = 0.85)
g.fig.suptitle("Mean Age per Dataset by Status", fontsize = 16)
g.fig.set_size_inches(25, 6)
plt.savefig("Age_Metadata_Per_Dataset_Per_Status.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Suppl_1A. Legend colors dataset.

legend_handles = [mpatches.Patch(color = color, label = dataset) for dataset, color in dataset_colors.items()]

plt.figure(figsize = (10, 6))
plt.legend(handles = legend_handles, title = "Dataset", bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.axis('off')
plt.tight_layout()
plt.savefig("Dataset_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Suppl_2B. Barplot dataset per region.

region_pivot = region_counts.pivot(index = "Region", columns = "Dataset", values = "Proportion").fillna(0)

region_pivot = region_pivot[my_levels]

fig, ax = plt.subplots(figsize = (7,6))
bar_width = 0.5

bottom = np.zeros(len(region_pivot))

for dataset in my_levels:
    ax.bar(
        region_pivot.index, 
        region_pivot[dataset], 
        bottom = bottom, 
        color = dataset_colors[dataset], 
        label = dataset
    )
    bottom += region_pivot[dataset].values

ax.set_ylabel("Proportion of Samples")
ax.set_xlabel("Region")
ax.set_title("Proportion of Each Dataset per Region (Stacked)")
ax.set_xticklabels(region_pivot.index, rotation = 45, ha = "right")
ax.legend(title = "Dataset", bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.tight_layout()
plt.savefig("Dataset_Proportions_Per_Region_Stacked.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_1E_1G. Umap Basic_Clusters Oligos OPCs.

adata_sc = adata[adata.obs["Region"] == "Spinal_Cord"].copy()

my_levels = ["CyPs", "RG", 
             "Early_OPCs", "Late_OPCs", 
             "Early_COPs", "Late_COPs", 
             "Oligos_OPALIN+", "Oligos_RBFOX1+"]

my_colors = ["#AB63FA", "#E41A1C", 
             "#8DA0CB", "#E78AC3", 
             "#A6D854", "#FFD92F", 
             "#4B0082", "#D95F02"]

adata_sc.obs["Basic_Clusters"] = adata_sc.obs["Basic_Clusters"].astype("category")
adata_sc.obs["Basic_Clusters"] = adata_sc.obs["Basic_Clusters"].cat.set_categories(my_levels)
adata_sc.obs["cell_type"] = adata_sc.obs["Basic_Clusters"]

fig = sc.pl.umap(
    adata_sc,
    color = "Basic_Clusters",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(15, 15)
fig.patch.set_facecolor("white")

ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_3_MC_SC_Oligos_OPCs_Basic_Clusters.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white')

In [ ]:
### Main. Bar Basic_Clusters Colors.

colors = ["#AB63FA", "#E41A1C", 
          "#8DA0CB", "#E78AC3", 
          "#A6D854", "#FFD92F", 
          "#4B0082", "#D95F02"]

fig, ax = plt.subplots(figsize=(6, 2))

left = 0
for color in colors:
    ax.barh(0, 1, left = left, color = color, edgecolor = "none")
    left += 1

ax.set_xlim(0, len(colors))
ax.axis("off")

plt.tight_layout()
plt.savefig("Horizontal_Bar_Basic_Clusters.png", dpi = 800)
plt.show()

In [ ]:
### Main Legend Plot Oligos_OPALIN+ and Oligos_RBFOX1+.

clusters = ["Oligos_OPALIN+", "Oligos_RBFOX1+"]
colors = ["#4B0082", "#D95F02"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 2))
plt.legend(
    handles = legend_handles,
    title = "Basic_Clusters",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Oligos_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main Legend Plot Basic_Clusters_Oligos_OPCs.

clusters = ["CyPs", "RG", 
            "Early_OPCs", "Late_OPCs", 
            "Early_COPs", "Late_COPs", 
            "Oligos_OPALIN+", "Oligos_RBFOX1+"]

colors = ["#AB63FA", "#E41A1C", 
          "#8DA0CB", "#E78AC3", 
          "#A6D854", "#FFD92F", 
          "#4B0082", "#D95F02"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (6, 4))
plt.legend(
    handles = legend_handles,
    title = "Basic_Clusters",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Basic_Clusters_Oligos_OPCs_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main Legend Plot Oligos_OPALIN+, Oligos_RBFOX1+ and Shared for Go.

clusters = ["Oligos_OPALIN+", "Oligos_RBFOX1+", "Shared"]
colors = ["#4B0082", "#D95F02", "red"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 2))
plt.legend(
    handles = legend_handles,
    title = "Basic_Clusters",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Oligos_Basic_Clusters_Shared_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Proportions Basic_Clusters per Region

df = adata.obs.copy()
df = df[df["Annotation"].isin(["Oligodendrocytes"])]
df["Annotation"] = df["Annotation"].astype(str).copy()
df = df[df["Basic_Clusters"].isin(["Oligos_OPALIN+", "Oligos_RBFOX1+"])]

df_grouped = (
    df.groupby(["Sample_ID", "Region", "Basic_Clusters", "Annotation"])
    .size()
    .reset_index(name = "count")
)

df_wide = df_grouped.pivot_table(
    index = ["Sample_ID", "Region", "Annotation"],
    columns = "Basic_Clusters",
    values = "count",
    fill_value = 0
).reset_index()

df_wide["total"] = df_wide["Oligos_OPALIN+"] + df_wide["Oligos_RBFOX1+"]
df_long = df_wide.melt(
    id_vars = ["Sample_ID", "Region", "Annotation", "total"],
    value_vars = ["Oligos_OPALIN+", "Oligos_RBFOX1+"],
    var_name = "Subpopulation",
    value_name = "count"
)
df_long["prop"] = df_long["count"] / df_long["total"]

color_map = {"Oligos_OPALIN+": "#4B0082", "Oligos_RBFOX1+": "#D95F02"}

g = sns.FacetGrid(
    df_long,
    col = "Region",
    sharex = False,
    sharey = True,
    height = 4,
    aspect = 1.2
)

g.map_dataframe(
    sns.violinplot,
    x = "Annotation", y = "prop", hue = "Subpopulation",
    palette = color_map,
    cut = 0, inner = None, dodge = True, alpha = 0.3,
    linewidth = 0
)

g.map_dataframe(
    sns.stripplot,
    x = "Annotation", y = "prop", hue = "Subpopulation",
    palette = color_map,
    dodge = True, jitter = 0.15, size = 4, alpha = 0.8, edgecolor = "black", linewidth = 0
)

for ax in g.axes.flatten():
    ax.tick_params(axis = "x", rotation = 45)

handles, labels = g.axes[0][0].get_legend_handles_labels()
g.fig.legend(handles[:2], labels[:2], title = "Subpopulation", loc = "upper center", ncol = 2)

g.set_axis_labels("Annotation", "Proportion", fontweight = "bold")
g.set_titles(col_template = "{col_name}")
sns.despine(trim=True)

plt.subplots_adjust(top = 0.8)
g.fig.suptitle("Proportion of Oligos Subpopulations", fontsize = 14, fontweight = "bold")
g.fig.set_size_inches(4, 5)
plt.savefig("Proportion_Oligos_Subpopulations.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Proportions Basic_Clusters per Age_Group.

df = adata.obs[["Sample_ID", "Basic_Clusters", "Age", "Region", "Status", "Enrichment"]].copy()
df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
df = df.dropna(subset=["Age"])
#df = df[(df["Status"] == "Control") & (df["Enrichment"] == "No")].copy()

bins = np.arange(25, 110, 15)
labels = [f"{int(b)}-{int(b + 14)}" for b in bins[:-1]]
df["AgeGroup"] = pd.cut(df["Age"], bins = bins, labels = labels, right = False)

df_grouped = (
    df.groupby(["Sample_ID", "Region", "AgeGroup", "Basic_Clusters"])
    .size()
    .reset_index(name = "count")
)

df_wide = df_grouped.pivot_table(
    index = ["Sample_ID", "Region", "AgeGroup"],
    columns = "Basic_Clusters",
    values = "count",
    fill_value = 0
).reset_index()

cluster_cols = [c for c in df_wide.columns if c not in ["Sample_ID", "Region", "AgeGroup"]]
df_wide["total"] = df_wide[cluster_cols].sum(axis = 1)
for c in cluster_cols:
    df_wide[c + "_prop"] = df_wide[c] / df_wide["total"]

df_long = df_wide.melt(
    id_vars = ["Sample_ID", "Region", "AgeGroup"],
    value_vars = [c + "_prop" for c in cluster_cols],
    var_name = "Cluster",
    value_name = "Proportion"
)

df_long["Cluster"] = df_long["Cluster"].str.replace("_prop", "")

color_map = {"Oligos_OPALIN+": "#4B0082", "Oligos_RBFOX1+": "#D95F02"}

g = sns.FacetGrid(df_long, col = "Region",
    sharex = True, sharey = True, height = 4, aspect = 1.2
)

g.map_dataframe(
    sns.violinplot, x = "AgeGroup", y = "Proportion", hue = "Cluster",
    palette = color_map, cut = 0, inner = None, dodge = True, alpha = 0.3
)

g.map_dataframe(
    sns.stripplot, x = "AgeGroup", y = "Proportion", hue = "Cluster",
    palette = color_map, dodge = True, jitter = 0.15, size = 4, alpha = 0.8, edgecolor = "black", linewidth = 0
)

for ax in g.axes.flatten():
    ax.tick_params(axis = "x", rotation = 45)

handles, labels = g.axes[0][0].get_legend_handles_labels()
g.fig.legend(handles[:2], labels[:2], title = "Cluster", loc = "upper center", ncol = 2)

g.set_axis_labels("Age Group (years)", "Proportion of Cells", fontweight = "bold")
g.set_titles(col_template = "{col_name}")
sns.despine(trim = True)

plt.subplots_adjust(top=0.85)
g.fig.suptitle("Proportion of Cells per Cluster Across Age Groups by Region", fontsize = 14, fontweight = "bold")

g.fig.set_size_inches(7, 4)
plt.savefig("Main_1_Proportion_Oligos_Basic_Clusters_per_Age_Group.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Proportions Basic_Clusters per Age_Group.

df = adata.obs[["Sample_ID", "Basic_Clusters", "Sex", "Region", "Status", "Enrichment"]].copy()
#df = df[(df["Status"] == "Control") & (df["Enrichment"] == "No")].copy()

df_grouped = (
    df.groupby(["Sample_ID", "Region", "Sex", "Basic_Clusters"])
    .size()
    .reset_index(name = "count")
)

df_wide = df_grouped.pivot_table(
    index = ["Sample_ID", "Region", "Sex"],
    columns = "Basic_Clusters",
    values = "count",
    fill_value = 0
).reset_index()

cluster_cols = [c for c in df_wide.columns if c not in ["Sample_ID", "Region", "Sex"]]
df_wide["total"] = df_wide[cluster_cols].sum(axis = 1)
for c in cluster_cols:
    df_wide[c + "_prop"] = df_wide[c] / df_wide["total"]

df_long = df_wide.melt(
    id_vars = ["Sample_ID", "Region", "Sex"],
    value_vars = [c + "_prop" for c in cluster_cols],
    var_name = "Cluster",
    value_name = "Proportion"
)

df_long["Cluster"] = df_long["Cluster"].str.replace("_prop", "")

color_map = {"Oligos_OPALIN+": "#4B0082", "Oligos_RBFOX1+": "#D95F02"}

g = sns.FacetGrid(df_long, col = "Region",
    sharex = True, sharey = True, height = 4, aspect = 1.2
)

g.map_dataframe(
    sns.violinplot, x = "Sex", y = "Proportion", hue = "Cluster",
    palette = color_map, cut = 0, inner = None, dodge = True, alpha = 0.3
)

g.map_dataframe(
    sns.stripplot, x = "Sex", y = "Proportion", hue = "Cluster",
    palette = color_map, dodge = True, jitter = 0.15, size = 4, alpha = 0.8, edgecolor = "black", linewidth = 0
)

for ax in g.axes.flatten():
    ax.tick_params(axis = "x", rotation = 45)

handles, labels = g.axes[0][0].get_legend_handles_labels()
#g.fig.legend(handles[:2], labels[:2], title = "Cluster", loc = "upper center", ncol = 2)

g.set_axis_labels("Sex", "Proportion of Cells", fontweight = "bold")
g.set_titles(col_template = "{col_name}")
sns.despine(trim = True)

plt.subplots_adjust(top=0.85)
g.fig.suptitle("", fontsize = 14, fontweight = "bold")

g.fig.set_size_inches(3, 3)
plt.savefig("Main_1_Proportion_Oligos_Basic_Clusters_per_Sex.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Suppl_1D. Barplot Annotation per Dataset per region

proportions = (
    meta_data.groupby(["Dataset", "Region", "Annotation"])
    .size()
    .reset_index(name = "Count")
)

proportions["Proportion"] = (
    proportions.groupby(["Dataset", "Region"])["Count"].transform(lambda x: x / x.sum())
)

proportions["Annotation"] = pd.Categorical(proportions["Annotation"], categories = my_levels, ordered = True)
proportions["Dataset"] = proportions["Dataset"].astype(str)

region_dfs = {region: df for region, df in proportions.groupby("Region")}

regions = list(region_dfs.keys())
ncols = min(len(regions), 3)
nrows = (len(regions) + ncols - 1) // ncols

fig, axes = plt.subplots(nrows = nrows, ncols = ncols, figsize = (10, 6), sharey = True)
axes = axes.flatten()

for ax, (region, subset) in zip(axes, region_dfs.items()):
    subset = subset.groupby("Dataset").filter(lambda x: x["Count"].sum() > 0)
    
    datasets = subset["Dataset"].unique()
    datasets = sorted(datasets)  
    x = range(len(datasets))
    bottom = pd.Series(0, index = datasets)

    for annotation in my_levels:
        ann_data = subset[subset["Annotation"] == annotation]
        values = ann_data.set_index("Dataset")["Proportion"].reindex(datasets, fill_value = 0)
        
        ax.bar(
            x, values, bottom = bottom,
            color = color_map[annotation],
            width = 0.8,
            label = annotation if region == regions[0] else ""
        )
        bottom += values

    ax.set_xticks(x)
    ax.set_xticklabels(datasets, rotation = 45, ha = 'right', fontsize = 9)
    ax.set_xlabel("")
    ax.set_ylabel("Proportion")
    ax.set_facecolor("white")
for ax in axes[len(regions):]:
    ax.remove()

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, title = "Annotation", loc = "upper center", ncol = 4, frameon = False, bbox_to_anchor = (0.5, 1.1))
plt.tight_layout(rect = [0, 0, 1, 0.95])
plt.savefig("Suppl_1_MC_SC_Proportions_Annotation_Dataset_Region.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_2A. Umap Subclusters OPCs.

my_levels = ["CyPs", "Radial_Glia", "Early_OPCs", "Late_OPCs", "Immune_OPCs",
             "Early_COPs", "Late_COPs"]

my_colors = ["#AB63FA", "#E41A1C", 
          "#8DA0CB", "#E78AC3", "#C1442E",
          "#A6D854", "#FFD92F"]

adata.obs["Subclusters"] = adata.obs["Subclusters"].astype("category")
adata.obs["Subclusters"] = adata.obs["Subclusters"].cat.set_categories(my_levels)
adata.obs["cell_type"] = adata.obs["Subclusters"]

fig = sc.pl.umap(
    adata,
    color = "Subclusters",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(12, 12)
fig.patch.set_facecolor("white")

ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_2_MC_SC_Subclusters_OPCs.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white')

In [ ]:
### Main Legend Plot Subclusters_OPCs.

clusters = ["CyPs", "RG", 
            "Early_OPCs", "Immune_OPCs", "Late_OPCs", 
            "Early_COPs", "Late_COPs"]

colors = ["#AB63FA", "#E41A1C", 
          "#8DA0CB", "#C1442E", "#E78AC3", 
          "#A6D854", "#FFD92F"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (6, 4))
plt.legend(
    handles = legend_handles,
    title = "Subclusters",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Subclusters_OPCs_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main. Bar Basic_Clusters Colors.

colors = ["#AB63FA", "#E41A1C", 
          "#8DA0CB", "#C1442E", "#E78AC3", 
          "#A6D854", "#FFD92F"]

fig, ax = plt.subplots(figsize=(6, 2))

left = 0
for color in colors:
    ax.barh(0, 1, left = left, color = color, edgecolor = "none")
    left += 1

ax.set_xlim(0, len(colors))
ax.axis("off")

plt.tight_layout()
plt.savefig("Horizontal_Bar_OPCs_Subclusters.png", dpi = 800)
plt.show()

In [ ]:
### Main_2B. Barplot Subclusters OPCs per region.

df = adata.obs[["Region", "Subclusters"]].copy()

region_counts = (
    df.groupby(["Region", "Subclusters"])
    .size()
    .reset_index(name="Count")
)

region_counts["Proportion"] = (
    region_counts.groupby("Region")["Count"].transform(lambda x: x / x.sum())
)

region_pivot = (
    region_counts.pivot(index = "Region", columns = "Subclusters", values = "Proportion")
    .fillna(0)
)

my_levels = ["CyPs", "Radial_Glia", "Early_OPCs", "Late_OPCs", 
             "Immune_OPCs", "Early_COPs", "Late_COPs"]

subclusters_colors = ["#AB63FA", "#E41A1C", "#8DA0CB", 
                      "#E78AC3", "#C1442E", "#A6D854", "#FFD92F"]

region_pivot = region_pivot[my_levels]

fig, ax = plt.subplots(figsize = (7, 6))
bottom = np.zeros(len(region_pivot))

for i, sub in enumerate(my_levels):
    ax.bar(
        region_pivot.index,
        region_pivot[sub],
        bottom = bottom,
        color = subclusters_colors[i],
        label = sub
    )
    bottom += region_pivot[sub].values

ax.set_ylabel("Proportion of Cells")
ax.set_xlabel("Region")
ax.set_title("Proportion of Each Subcluster per Region (Stacked)")
ax.set_xticklabels(region_pivot.index, rotation = 45, ha = "right")
ax.legend(title = "Subcluster", bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.tight_layout()
plt.savefig("Main_2_Subcluster_Proportions_Per_Region_Stacked.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_2A. Umap Subclusters Oligodendrocytes.

my_levels = ["Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5",
             "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+"]

my_colors = ["#FF6F61", "#6B5B95", 
             "#88B04B", "#D85C7A", 
             "#00CED1",
             "#FFD700", "#9B2335"]


adata.obs["Subclusters"] = adata.obs["Subclusters"].astype("category")
adata.obs["Subclusters"] = adata.obs["Subclusters"].cat.set_categories(my_levels)
adata.obs["cell_type"] = adata.obs["Subclusters"]

fig = sc.pl.umap(
    adata,
    color = "Subclusters",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(12, 12)
fig.patch.set_facecolor("white")

ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_2_MC_SC_Subclusters_Oligos.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white')

In [ ]:
### Main Legend Plot Subclusters Oligos.

clusters = ["Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5",
             "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+"]

colors = ["#FF6F61", "#6B5B95", 
             "#88B04B", "#D85C7A", 
             "#00CED1",
             "#FFD700", "#9B2335"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 4))
plt.legend(
    handles = legend_handles,
    title = "",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Oligos_Subclusters_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_2B. Barplot Subclusters Oligos per region.

df = adata.obs[["Region", "Subclusters"]].copy()

region_counts = (
    df.groupby(["Region", "Subclusters"])
    .size()
    .reset_index(name="Count")
)

region_counts["Proportion"] = (
    region_counts.groupby("Region")["Count"].transform(lambda x: x / x.sum())
)

region_pivot = (
    region_counts.pivot(index = "Region", columns = "Subclusters", values = "Proportion")
    .fillna(0)
)

my_levels = ["Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5",
             "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+"]

subclusters_colors = ["#FF6F61", "#6B5B95", 
             "#88B04B", "#D85C7A", 
             "#00CED1",
             "#FFD700", "#9B2335"]

region_pivot = region_pivot[my_levels]

fig, ax = plt.subplots(figsize = (7, 6))
bottom = np.zeros(len(region_pivot))

for i, sub in enumerate(my_levels):
    ax.bar(
        region_pivot.index,
        region_pivot[sub],
        bottom = bottom,
        color = subclusters_colors[i],
        label = sub
    )
    bottom += region_pivot[sub].values

ax.set_ylabel("Proportion of Cells")
ax.set_xlabel("Region")
ax.set_title("Proportion of Each Subcluster per Region (Stacked)")
ax.set_xticklabels(region_pivot.index, rotation = 45, ha = "right")
ax.legend(title = "Subcluster", bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.tight_layout()
plt.savefig("Main_2_Subcluster_Oligos_Proportions_Per_Region_Stacked.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
matrix = pd.crosstab(adata.obs["Subclusters"], adata.obs["Region"])
matrix

In [ ]:
### Main 2. Oligos_OPCs Superclusters

my_levels = ["RG", "CyPs", "Reactive_OPCs", "Homeostatic_OPCs", "Reactive_Oligos", "Homeostatic_Oligos"]

my_colors = ["#AB63FA", "#E41A1C", "#ff7f0e", "#17becf", 
             "#e377c2", "#1f77b4"]

adata.obs["Superclusters"] = adata.obs["Superclusters"].astype("category")
adata.obs["Superclusters"] = adata.obs["Superclusters"].cat.set_categories(my_levels)
adata.obs["cell_type"] = adata.obs["Superclusters"]

fig = sc.pl.umap(
    adata,
    color = "Superclusters",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(20, 20)
fig.patch.set_facecolor("white")

ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_2_MC_SC_Oligos_OPCs_Superclusters.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white')

In [ ]:
### Main 2. Oligos Superclusters

my_levels = ["Reactive_Oligos", "Homeostatic_Oligos"]

my_colors = ["#e377c2", "#1f77b4"]

adata.obs["Superclusters"] = adata.obs["Superclusters"].astype("category")
adata.obs["Superclusters"] = adata.obs["Superclusters"].cat.set_categories(my_levels)
adata.obs["cell_type"] = adata.obs["Superclusters"]

fig = sc.pl.umap(
    adata,
    color = "Superclusters",
    palette = my_colors,
    size = 20,
    legend_loc = None,
    frameon = False,
    title = "",
    return_fig = True
)

fig.set_size_inches(15, 15)
fig.patch.set_alpha(0.0)
ax.set_facecolor("none")
ax = fig.axes[0]

patches = [
    Line2D([0], [0], marker = 'o', color = 'w', label = label,
           markerfacecolor = color, markersize = 10)
    for label, color in zip(my_levels, my_colors)
]

ax.legend(
    handles = patches,
    loc = 'center left',
    bbox_to_anchor = (1.05, 0.5),
    fontsize = 12,
    frameon = False,
    borderaxespad = 0
)

fig.savefig("Main_2_MC_SC_Oligos_Superclusters.png", dpi = 800, bbox_inches = 'tight', facecolor = 'white', transparent = True)

In [ ]:
### Main Legend Plot Superclusters Oligos OPCs.

clusters = ["RG", "CyPs", "Reactive_OPCs", "Homeostatic_OPCs", "Reactive_Oligos", "Homeostatic_Oligos"]

colors = ["#AB63FA", "#E41A1C", "#ff7f0e", "#17becf", 
             "#e377c2", "#1f77b4"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 4))
plt.legend(
    handles = legend_handles,
    title = "",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("Oligos_OPCs_Superclusters_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_1. Basic CLusters Oligos OPCs markers

marker_genes = [
    "VCAN", "PDGFRA",
    "ST18", "LRP2",
    "HELLS", "CENPK",
    "ELL2", "GAP43",
    "TPST1", "HIF3A",
    "OLIG1", "OLIG2",
    "GPR17", "BCAS1",
    "SEMA4D", "FRY",
    "OPALIN", "ANK3",
    "RBFOX1", "RASGRF1"
]

my_levels = [
    "CyPs", "RG", "Early_OPCs", "Late_OPCs",
    "Early_COPs", "Late_COPs", "Oligos_OPALIN+", "Oligos_RBFOX1+"
]

adata.obs["Basic_Clusters"] = adata.obs["Basic_Clusters"].astype("category")
adata.obs["Basic_Clusters"] = adata.obs["Basic_Clusters"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names=marker_genes[::-1],
    groupby = "Basic_Clusters",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize=12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(8, 8)
dot.fig.tight_layout()

dot.fig.savefig("Main_1_Oligos_OPCs_Basic_Clusters_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2. Oligodendrocyte Lineage Markers per Subcluster.

marker_genes = [
    "OPALIN", "NLGN1", "CTNND2",
    "RBFOX1", "RASGRF1", "AFF3",
    "CNP", "MBP", "S100B"
]

my_levels = [
    "Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5",
    "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+"
]

adata.obs["Subclusters"] = adata.obs["Subclusters"].astype("category")
adata.obs["Subclusters"] = adata.obs["Subclusters"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(6, 3.5)
dot.fig.tight_layout()

dot.fig.savefig("Main_1_Oligos_Subclusters_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2. Oligodendrocyte Subcluster Markers.

marker_genes = [
    "FRY", "SGCZ", 
    "PLXDC2", "SVEP1",
    "SORL1", "ACTN2",
    "TMEM178A", "SCCPDH",
    "NRG3", "FLRT2",
    "BCL6", "IFI16"
]

my_levels = [
    "Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5",
    "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+"
]

adata.obs["Subclusters"] = adata.obs["Subclusters"].astype("category")
adata.obs["Subclusters"] = adata.obs["Subclusters"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(5, 4)
dot.fig.tight_layout()

dot.fig.savefig("Main_1_Oligos_Subclusters_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2. Oligodendrocyte Subcluster_Activation Markers Literature.

marker_genes = [
"APOD", "CD63", "CD81", "CD9", "CRYAB", "DYNLL1", "EIF1", "EIF5", "FTH1", "FTL", 
"HSP90AA1", "HSP90AB1", "HSPA5", "HSPA8",  "KLK6", 
"LGALS1", "PLOD3", "RPL13", "RPL13A", "RPL18", "RPL32", "RPL37", "RPL37A", "RPLP2", 
"RPS16"mat "RPS19", "RPS28", "RPSA",  "TXNRD1", "UBA52",
    
"FOS", "GADD45B", 
    "CAMK2D", "HSD17B7", "NLRC5", "STAT3", "ST8SIA5"  
]

my_levels = [
    "Reactive_Olig_1", "Reactive_Olig_2", "Reactive_Olig_3", "Reactive_Olig_4", "Reactive_Olig_5",
    "Reactive_Olig_Immune_OPALIN+", "Reactive_Olig_Immune_RBFOX1+",
    "Homeostatic_Olig_1", "Homeostatic_Olig_2", "Homeostatic_Olig_3", "Homeostatic_Olig_4", "Homeostatic_Olig_5",
    "Homeostatic_Olig_Immune_OPALIN+", "Homeostatic_Olig_Immune_RBFOX1+"
]

adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].astype("category")
adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters_Activation",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(6, 7)
dot.fig.tight_layout()

dot.fig.savefig("Main_2_Oligos_Subclusters_Activation_Markers_Literature.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2. OPC Subcluster Markers.

marker_genes = [
    "HELLS", "CENPK", 
    "ELL2", "GAP43",
    "TPST1", "SAMD4A",
     "HERC6", "IFI6",
    "OLIG1", "OLIG2",
    "GPR17", "BCAS1",
    "FRY", "SEMA4D"
]

my_levels = [
    "CyPs", "RG", "Early_OPCs", "Immune_OPCs", "Late_OPCs", "Early_COPs", "Late_COPs"
]

adata.obs["Subclusters"] = adata.obs["Subclusters"].astype("category")
adata.obs["Subclusters"] = adata.obs["Subclusters"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(5, 4)
dot.fig.tight_layout()

dot.fig.savefig("Main_2_OPCs_Subclusters_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2.OPC Subcluster and Activation Markers.

marker_genes = [
    "TPST1", "HERC6", "OLIG1", "GPR17", "SEMA4D", "APOD", "FTH1", "FTL", "CD9", "CD63", "MARCKSL1" 
]

my_levels = [
    "Reactive_Early_OPCs", "Reactive_Immune_OPCs", "Reactive_Late_OPCs", "Reactive_Early_COPs", "Reactive_Late_COPs",
    "Homeostatic_Early_OPCs", "Homeostatic_Immune_OPCs", "Homeostatic_Late_OPCs", "Homeostatic_Early_COPs", "Homeostatic_Late_COPs",
]

adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].astype("category")
adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters_Activation",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(5, 4)
dot.fig.tight_layout()

dot.fig.savefig("Main_2_OPCs_Subclusters_Activation_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
### Main_2. Oligos Subcluster and Activation Markers.

marker_genes = [
    "FRY","PLXDC2", "SORL1", "TMEM178A", "FLRT2", "BCL6" , "APOD", "FTH1", "FTL", "CD9", "CD63", "MARCKSL1" 
]

my_levels = [
    "Reactive_Olig_1", "Reactive_Olig_2", "Reactive_Olig_3", "Reactive_Olig_4", "Reactive_Olig_5",
    "Reactive_Olig_Immune_OPALIN+", "Reactive_Olig_Immune_RBFOX1+",
    "Homeostatic_Olig_1", "Homeostatic_Olig_2", "Homeostatic_Olig_3", "Homeostatic_Olig_4", "Homeostatic_Olig_5",
    "Homeostatic_Olig_Immune_OPALIN+", "Homeostatic_Olig_Immune_RBFOX1+"
]

adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].astype("category")
adata.obs["Subclusters_Activation"] = adata.obs["Subclusters_Activation"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata,
    var_names = marker_genes[::-1],
    groupby = "Subclusters_Activation",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(5, 4)
dot.fig.tight_layout()

dot.fig.savefig("Main_2_Oligos_Subclusters_Activation_Markers.png", dpi = 800, bbox_inches = "tight")

In [ ]:
clusters = ["RG", "CyPs", "Reactive_OPCs", "Homeostatic_OPCs", "Reactive_Oligos", "Homeostatic_Oligos"]

colors = ["#AB63FA", "#E41A1C", "#ff7f0e", "#17becf", 
             "#e377c2", "#1f77b4"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 4))
plt.legend(
    handles = legend_handles,
    title = "",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("OPCs_Subclusters_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
clusters = ["CyPs", "RG", "Early_OPCs", "Immune_OPCs", "Late_OPCs", "Early_COPs", "Late_COPs"]

colors = ["#AB63FA", "#E41A1C", 
         "#8DA0CB", "#C1442E", "#E78AC3", 
         "#A6D854", "#FFD92F"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 4))
plt.legend(
    handles = legend_handles,
    title = "",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("OPCs_Subclusters_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main Legend Plot Reactive_OPCs and Homeostatic_OPCs.

clusters = ["Reactive_OPCs", "Homeostatic_OPCs"]
colors = ["#ff7f0e", "#17becf"]

legend_handles = [
    mpatches.Patch(color = color, label = cluster)
    for cluster, color in zip(clusters, colors)
]

plt.figure(figsize = (4, 2))
plt.legend(
    handles = legend_handles,
    title = "Basic_Clusters",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left',
    frameon = False
)
plt.axis("off")
plt.tight_layout()
plt.savefig("OPCs_Superclusters_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_4. Legend Plot Dataset.

os.chdir("/staging/leuven/stg_00077/projects/maria/Maria_Projects/Single_cell_sequencing_2022/Data/Datasets/01_Merged/Adata_Objects")
my_levels = [
    "Bakken_2021_MC_Control",
    "Georgopoulou_2025_MC_Control_ALS",
    "Li_2023_MC_Control_ALS",
    "Limone_2024_MC_Control_ALS",
    "ONeill_2024_MC_Control_ALS_FTD",
    "Pineda_2024_MC_Control_ALS",
    "Prashant_2024_MC_Control",
    "Silleti_2023_MC_Control",
    "Velmeshev_2023_MC_Control",
    "Zemke_2023_MC_Control",
    "Donovan_2024_SC_Control",
    "Gautier_2023_SC_Control",
    "Georgopoulou_2025_SC_Control_ALS",
    "Silleti_2023_SC_Control",
    "Tansley_2022_SC_Control",
    "Yadav_2023_SC_Control",
    "Zelic_2025_SC_Control_ALS",
    "Zhang_2021_SC_Control"
]

cols = [
    "#4B9E9E",  # Bakken
    "#D9B44A",  # Georgopoulou MC
    "#A892B4",  # Li
    "#8AA96B",  # Limone
    "#5067C5",  # ONeill
    "#C95D63",  # Pineda
    "#FF6F61",  # Prashant
    "#6A5ACD",  # Silleti MC
    "#20B2AA",  # Velmeshev
    "#D97D54",  # Zemke

    "#77DD77",  # Donovan
    "#E07B39",  # Gautier
    "#C48B9F",  # Georgopoulou SC
    "#613659",  # Silleti SC
    "#3BA7CB",  # Tansley
    "#FFD700",  # Yadav
    "#708090",  # Zelic
    "#4A90A4"   # Zhang
]

dataset_colors = dict(zip(my_levels, cols))

legend_handles = [mpatches.Patch(color = color, label = dataset) for dataset, color in dataset_colors.items()]

plt.figure(figsize = (10, 6))
plt.legend(
    handles = legend_handles,
    title = "Dataset",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left'
)
plt.axis('off')
plt.tight_layout()
plt.savefig("Main_4_Dataset_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main 4. Bar Dataset Colors.

colors = ["#4B9E9E",  # Bakken
    "#D9B44A",  # Georgopoulou MC
    "#A892B4",  # Li
    "#8AA96B",  # Limone
    "#5067C5",  # ONeill
    "#C95D63",  # Pineda
    "#FF6F61",  # Prashant
    "#6A5ACD",  # Silleti MC
    "#20B2AA",  # Velmeshev
    "#D97D54",  # Zemke

    "#77DD77",  # Donovan
    "#E07B39",  # Gautier
    "#C48B9F",  # Georgopoulou SC
    "#613659",  # Silleti SC
    "#3BA7CB",  # Tansley
    "#FFD700",  # Yadav
    "#708090",  # Zelic
    "#4A90A4"   # Zhang]
         ]

fig, ax = plt.subplots(figsize = (6, 2))

left = 0
for color in colors:
    ax.barh(0, 1, left = left, color = color, edgecolor = "none")
    left += 1

ax.set_xlim(0, len(colors))
ax.axis("off")

plt.tight_layout()
plt.savefig("Horizontal_Bar_Dataset.png", dpi = 800)
plt.show()

In [ ]:
### Main 4. Bar Status Colors.

colors = ["#B1746FFF",
          "#ADB17DFF",
          "#5B8FA8FF"
         ]

fig, ax = plt.subplots(figsize = (6, 2))

left = 0
for color in colors:
    ax.barh(0, 1, left = left, color = color, edgecolor = "none")
    left += 1

ax.set_xlim(0, len(colors))
ax.axis("off")

plt.tight_layout()
plt.savefig("Horizontal_Bar_Status.png", dpi = 800)
plt.show()

In [ ]:
### Legend Status.

my_levels = [
    "C9ALS",
    "Control",
    "sALS"
]

cols = ["#B1746FFF",
          "#ADB17DFF",
          "#5B8FA8FF"
]

dataset_colors = dict(zip(my_levels, cols))

legend_handles = [mpatches.Patch(color = color, label = dataset) for dataset, color in dataset_colors.items()]

plt.figure(figsize = (10, 6))
plt.legend(
    handles = legend_handles,
    title = "Dataset",
    bbox_to_anchor = (1.05, 1),
    loc = 'upper left'
)
plt.axis('off')
plt.tight_layout()
plt.savefig("Main_4_Status_Legend.png", dpi = 800, bbox_inches = "tight")
plt.show()

In [ ]:
### Main_4. DEGs Number Bar Plot Oligos.

data1 = [11, 0]
data2 = [284, 152]
labels = ['C9ALS Vs Control', 'sALS Vs Control']
colors = ['#B1746FFF', '#5B8FA8FF']

fig, axes = plt.subplots(1, 2, figsize=(11, 1.5), sharex = True)

axes[0].barh(labels, data1, color = colors)
axes[0].invert_yaxis()

axes[1].barh(labels, data2, color = colors)
axes[1].invert_yaxis()

for ax in axes:
    ax.set_xlim(0, max(data2) + 20)
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.tick_params(left = False, bottom = False)
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.grid(False)

plt.tight_layout(pad = 1)

plt.savefig("Main_4_DEG_Number_Plot.png", dpi = 800, bbox_inches = 'tight', transparent = True)
plt.show()

In [ ]:
### Suppl_2. DEGs Number Bar Plot OPCs.

data1 = [48, 0]
data2 = [302, 144]
labels = ['C9ALS Vs Control', 'sALS Vs Control']
colors = ['#B1746FFF', '#5B8FA8FF']

fig, axes = plt.subplots(2, 1, figsize = (4, 1.5), sharex = True)

axes[0].barh(labels, data1, color = colors)
axes[0].invert_yaxis()

axes[1].barh(labels, data2, color = colors)
axes[1].invert_yaxis()

for ax in axes:
    ax.set_xlim(0, max(data2) + 20)
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.tick_params(left = False, bottom = False)
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.grid(False)

plt.tight_layout(pad = 1)

plt.savefig("Suppl_2_DEG_Number_Plot.png", dpi = 800, bbox_inches = 'tight', transparent = True)
plt.show()

In [ ]:
### Main_2 PPEF1 Feature Plot.

colors = ["#D5E5F7", "#2488F0", "#7F3F98", "#E22929", "#A81B1B"]
custom_cmap = LinearSegmentedColormap.from_list("custom", colors, N = 256)

sc.settings.set_figure_params(dpi = 800, figsize = (20, 20))
sc.pl.umap(
    adata_old,
    color = "PPEF1", 
    layer = "log1p_normalized",
    size = 20,
    cmap = custom_cmap,
    frameon = False,
    title = "",
    show = False
)

plt.gca().set_facecolor("white")
plt.gcf().set_facecolor("white")
plt.savefig("Main_2_PPEF1_MC_SC.png", dpi = 800, bbox_inches = "tight", facecolor = "white")

plt.show()

In [ ]:
### ### Main_2 PPEF1 DotPlot.

adata_old.obs["Subclusters_Annotation"] = np.where(
    adata_old.obs["Annotation"] == "Oligodendrocytes",
    adata_old.obs["Subclusters"],
    adata_old.obs["Annotation"]
)

marker_genes = ['PPEF1']


my_levels = [
    "Olig_1", "Olig_2", "Olig_3", "Olig_4", "Olig_5", "Olig_Immune_OPALIN+", "Olig_Immune_RBFOX1+",
    "Neurons", "OPCs", "Astrocytes", "Microglia",
             "Macrophages", "Endothelial Cells", "Mural Cells", "Fibroblasts",
             "Ependymal Cells", "Immune Cells", "Schwann Cells"
]

adata_old.obs["Subclusters_Annotation"] = adata_old.obs["Subclusters_Annotation"].astype("category")
adata_old.obs["Subclusters_Annotation"] = adata_old.obs["Subclusters_Annotation"].cat.set_categories(my_levels)

dot = sc.pl.dotplot(
    adata_old,
    var_names = marker_genes[::-1],
    groupby = "Subclusters_Annotation",
    cmap = "Spectral_r",
    dot_max = 1.0,
    dot_min = 0.0,
    #standard_scale = "var",
    colorbar_title = "Expression",
    title = "Dot Plot of Cell Type Marker Genes",
    return_fig = True,
    show = False
)

dot = dot.swap_axes()

for key, ax in dot.get_axes().items():
    if isinstance(ax, matplotlib.axes.Axes):
        for coll in ax.collections:
            coll.set_edgecolor("none")
        for spine in ax.spines.values():
            spine.set_visible(False)
        ax.set_facecolor("white")
        for label in ax.get_yticklabels():
            label.set_fontweight("bold")
            label.set_color("#3A3A3A")
        ax.set_xlabel("Cell Type (Clusters)", fontsize = 12)
        ax.set_ylabel("Marker Genes", fontsize = 12)
        ax.tick_params(axis = "x", rotation = 45, labelsize = 10)
        ax.tick_params(axis = "y", labelsize = 10)

dot.fig.set_size_inches(9, 2)

dot.fig.tight_layout()
plt.savefig("Main_2_DotPlot_PPEF1_MC_SC.png", dpi = 800, bbox_inches = "tight", facecolor = "white")
plt.show()

In [ ]:
### Main 6. Degree Centrality.

df_metrics = pd.read_excel("Centrality_Comparison_MC_SC_Oligos_Control_SC_Oligos_C9ALS_sALS_DNA.xlsx")

degree_df = df_metrics.melt(
    id_vars = ['gene'], 
    value_vars = ['degree_control', 'degree_disease'], 
    var_name = 'condition', 
    value_name = 'degree'
)

degree_df['condition'] = degree_df['condition'].map({
    'degree_control': 'Control',
    'degree_disease': 'Disease'
})

degree_df['log_degree'] = np.log1p(degree_df['degree'])

color_map = {"Control": "#556B2F", "Disease": "#7F1734"}

plt.figure(figsize = (4, 4))
g = sns.violinplot(
    data = degree_df,
    x = "condition",
    y = "log_degree",
    palette = color_map,
    cut = 0,
    inner = None,
    alpha = 0.3
)

sns.pointplot(
    data = degree_df,
    x = 'condition',
    y = 'log_degree',
    estimator = np.mean,
    ci = None,
    join = False,
    markers = 'D',
    color = 'white',
    scale = 1.2
)

sns.stripplot(
    data = degree_df,
    x = "condition",
    y = "log_degree",
    palette = color_map,
    jitter = 0.15,
    size = 4,
    alpha = 0.8,
    edgecolor = "black",
    linewidth = 0
)

plt.title("Log-transformed Degree Centrality", fontweight = "bold")
plt.xlabel("")
plt.ylabel("log(1 + Degree)")
plt.grid(False)
sns.despine(trim = True)
plt.tight_layout()
plt.savefig("Main_6_Degree_DNA_Boxplot_Violin.png", dpi = 800, bbox_inches = "tight")
plt.show()